In [ ]:
# IMPORT & LOAD LIBRARY + DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Load Data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission_df = pd.read_csv("submission.csv")
print("Data Berhasil Terload")

# Simpan ID test_df untuk submission
test_ids = test_df['ID'] 

# Menambahkan kolos is_test di masing masing data
train_df['is_test'] = 0
test_df['is_test'] = 1

# Rename Electricity Consumption di train_df
train_df.rename(columns={'electricity_consumption_gwh': 'electricity_consumption'}, inplace=True)

# MERGING & RAPIHIN DATA TRAIN DAN TEST 

# Menggabungkan train_df dan test_df
test_df['electricity_consumption'] = np.nan
all_df = pd.concat([train_df.drop(columns=['ID']), test_df.drop(columns=['ID'])], ignore_index=True)

# Menampilkan informasi DataFrame
print("\n--- Kombinasi dua data (5 Baris Pertama) ---")
print(all_df.head())
print("\n--- Info Kombinasi dua data ---")
print(all_df.info())

# Mengconvert kolom 'date' ke tipe datetime
all_df['date'] = pd.to_datetime(all_df['date'])

# Sort & Tampilkan data berdasarkan tahunnya 
all_df = all_df.sort_values(by=['cluster_id', 'date']).reset_index(drop=True)
print("\n--- Data setelah diurutkan ---")
print(all_df.head())

Data Berhasil Terload

--- Data setelah diurutkan ---
        date cluster_id  electricity_consumption  temperature_2m_max  \
0 2014-01-01  cluster_1                  358.032                10.8   
1 2014-01-02  cluster_1                  386.908                10.7   
2 2014-01-03  cluster_1                  395.319                11.7   
3 2014-01-04  cluster_1                  378.062                10.6   
4 2014-01-05  cluster_1                  372.324                11.4   

   temperature_2m_min  apparent_temperature_max  apparent_temperature_min  \
0                 4.2                       5.5                       0.4   
1                 7.0                       6.6                       3.3   
2                 7.1                       7.7                       2.5   
3                 5.9                       7.5                       1.5   
4                 2.4                       7.3                      -1.6   

   sunshine_duration  daylight_duration  wind_spee

In [ ]:
# FEATURE ENGINEERING 

# Time Based Features
all_df['year'] = all_df['date'].dt.year
all_df['month'] = all_df['date'].dt.month
all_df['day'] = all_df['date'].dt.day
all_df['day_of_week'] = all_df['date'].dt.dayofweek
all_df['day_of_year'] = all_df['date'].dt.dayofyear
all_df['quarter'] = all_df['date'].dt.quarter

# Fitur Siklis
all_df['month_sin'] = np.sin(2 * np.pi * all_df['month'] / 12)
all_df['month_cos'] = np.cos(2 * np.pi * all_df['month'] / 12)
all_df['day_of_week_sin'] = np.sin(2 * np.pi * all_df['day_of_week'] / 7)
all_df['day_of_week_cos'] = np.cos(2 * np.pi * all_df['day_of_week'] / 7)
all_df['day_of_year_sin'] = np.sin(2 * np.pi * all_df['day_of_year'] / 365.25)
all_df['day_of_year_cos'] = np.cos(2 * np.pi * all_df['day_of_year'] / 365.25)